In [25]:
import pandas as pd
import numpy as np
import string
import re
import random 
import os
import csv

from gensim.models.word2vec import Word2Vec

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn import metrics
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

from imblearn.ensemble import BalancedRandomForestClassifier

import gensim
import gensim.downloader as gensim_api
from numpy import asarray
from numpy import savetxt
from numpy import loadtxt

# Tf-idf

In [30]:
auc_scores = []
pr_auc = []

dat = pd.read_csv("20220115_all_paragraphs_2020_added.csv")
annotated = pd.read_csv("All_annotated_data_round_1_low_pride.csv")
dat = dat.merge(annotated[["speech_par_id", "pred_class"]], how = "left", on = "speech_par_id")

for filename in os.listdir("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_low_pride/"):
    ## load the test set 
    file = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_low_pride/" + "/" + filename
    test = pd.read_csv(file)
    train = annotated[~annotated['speech_par_id'].isin(test["speech_par_id"])]
    test = test[test['speech_par_id'].isin(dat['speech_par_id'])]
    train = train[train['speech_par_id'].isin(dat['speech_par_id'])] # make sure annotated data and the original df match
    # pipeline BOW, tfidf and RF
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', RandomForestClassifier(n_estimators=5000, max_depth=3,max_features='sqrt', random_state=0, class_weight="balanced")),
                                                 ])
    ## Fit the model to the training set
    text_clf = text_clf.fit(train['text'], train['pred_class'])
    ## Predict out-of-sample on the test set and compute AUC
    preds = text_clf.predict_proba(test['text'])
    fpr_d2v, tpr_d2v, thresholds_d2v = metrics.roc_curve(test['true_class'], preds[:,1], pos_label=1)
    auc_scores = auc_scores + [metrics.auc(fpr_d2v, tpr_d2v)]
    #PR AUC
    lr_precision, lr_recall, _ = precision_recall_curve(test['true_class'],  preds[:,1].tolist())
    lr_auc = auc(lr_recall, lr_precision)
    pr_auc = pr_auc + [lr_auc]
    print("AUC: "+str(metrics.auc(fpr_d2v, tpr_d2v)))
    print("PR AUC: " + str(lr_auc))
    
print("Mean AUC: " + str(np.mean(auc_scores)))
print("Mean PR AUC: " + str(np.mean(pr_auc)))

confusion = confusion_matrix(test['true_class'],text_clf.predict(test['text']) )
print(confusion)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AUC: 0.7606150793650793
PR AUC: 0.22538458654838897
AUC: 0.7693452380952381
PR AUC: 0.2921091965331401
AUC: 0.7460411718131433
PR AUC: 0.17151312430308696
AUC: 0.7471298495645289
PR AUC: 0.12002953128571925
AUC: 0.7401029295328583
PR AUC: 0.23064686482466365
AUC: 0.7405753968253967
PR AUC: 0.20074563969967418
AUC: 0.7922604908946952
PR AUC: 0.23978691047830994
AUC: 0.8268012668250199
PR AUC: 0.26233175899211947
AUC: 0.7612103174603174
PR AUC: 0.2145353725074794
AUC: 0.8082937450514648
PR AUC: 0.22959949976398206
AUC: 0.7414885193982582
PR AUC: 0.11931945348770756
AUC: 0.8090277777777778
PR AUC: 0.21146706017174238
AUC: 0.7679563492063493
PR AUC: 0.1946538398097317
AUC: 0.7757936507936508
PR AUC: 0.26018457572705267
AUC: 0.6721100554235946
PR AUC: 0.16293522427223336
AUC: 0.8662905779889152
PR AUC: 0.32668879061548095
AUC: 0.7834520981789391
PR AUC: 0.2649517656026478
AUC: 0.821159936658749
PR AUC: 0.23289874503271285
AUC: 0.7917656373713381
PR AUC: 0.29585478363316187
AUC: 0.7673198733

In [31]:
with open("low_pride_pr_auc_tfidf.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in pr_auc:
        csvwriter.writerow([row])    
    
with open("low_pride_auc_tfidf.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in auc_scores:
        csvwriter.writerow([row])

# Local Word2Vec RF

In [52]:
auc_scores = []
pr_auc = []

dat = pd.read_csv("20220131_all_paragraphs_2020_added_missings_added.csv")
annotated = pd.read_csv("All_annotated_data_round_1_low_pride_20220209.csv")
dat = dat.merge(annotated[["speech_par_id", "pred_class"]], how = "left", on = "speech_par_id")
avg_embeddings = loadtxt('avg_embeddings.csv', delimiter=',')

for filename in os.listdir("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_low_pride/"):
    ## load the test set 
    file = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_low_pride/" + "/" + filename
    test = pd.read_csv(file)
    train = annotated[~annotated['speech_par_id'].isin(test["speech_par_id"])]
    test = test[test['speech_par_id'].isin(dat['speech_par_id'])]
    train = train[train['speech_par_id'].isin(dat['speech_par_id'])] # make sure annotated data and the original df match
    # attach train/test set to original df
    train_id = train['speech_par_id']
    test_id = test['speech_par_id']    
    dat['test'] = np.where(dat['speech_par_id'].isin(test_id), 1, 0)
    dat['train'] = np.where(dat['speech_par_id'].isin(train_id), 1, 0)
    # get the embeddings to train/test set
    train_vec = np.asarray([avg_embeddings[i] for i in dat[dat['speech_par_id'].isin(train_id)].index.tolist()])
    test_vec = np.asarray([avg_embeddings[i] for i in dat[dat['speech_par_id'].isin(test_id)].index.tolist()])
    ## Fit the model to the training set
    text_clf = RandomForestClassifier(n_estimators=5000, max_depth=10, random_state=0, max_features = 0.2, class_weight="balanced")
    text_clf.fit(train_vec,  dat[dat['train']==1].pred_class) # use the original df to match the order
    ## Predict out-of-sample on the test set and compute AUC
    preds = text_clf.predict_proba(test_vec)
    fpr_d2v, tpr_d2v, thresholds_d2v = metrics.roc_curve(dat[dat['test']==1].pred_class, preds[:,1], pos_label=1)
    auc_scores = auc_scores + [metrics.auc(fpr_d2v, tpr_d2v)]
    #PR AUC
    lr_precision, lr_recall, _ = precision_recall_curve(dat[dat['test']==1].pred_class,  preds[:,1].tolist())
    lr_auc = auc(lr_recall, lr_precision)
    pr_auc = pr_auc + [lr_auc]
    print("AUC: "+str(metrics.auc(fpr_d2v, tpr_d2v)))
    print("PR AUC: " + str(lr_auc))
    
print("Mean AUC: " + str(np.mean(auc_scores)))
print("Mean PR AUC: " + str(np.mean(pr_auc)))

confusion = confusion_matrix(dat[dat['test']==1].pred_class,text_clf.predict(test_vec) )
print(confusion)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AUC: 0.7962193190815519
PR AUC: 0.2452714731276421
AUC: 0.841765873015873
PR AUC: 0.3437750214520384
AUC: 0.7718253968253967
PR AUC: 0.14939514863686978
AUC: 0.7315476190476191
PR AUC: 0.2036877338582541
AUC: 0.8190815518606492
PR AUC: 0.26481856870786036
AUC: 0.8729216152019003
PR AUC: 0.3422544116413206
AUC: 0.8370942201108472
PR AUC: 0.27704317838743586
AUC: 0.8387767220902612
PR AUC: 0.3242568673905283
AUC: 0.8435273159144893
PR AUC: 0.3530411680109525
AUC: 0.757818685669042
PR AUC: 0.20803165080201466
AUC: 0.803147268408551
PR AUC: 0.16490232040518385
AUC: 0.8077988915281077
PR AUC: 0.2139843885123077
AUC: 0.8037410926365796
PR AUC: 0.1936424477080629
AUC: 0.8341250989707047
PR AUC: 0.2920422331587763
AUC: 0.7372327790973872
PR AUC: 0.14605246539784233
AUC: 0.8111638954869359
PR AUC: 0.2763073033538633
AUC: 0.7490079365079365
PR AUC: 0.1933868744933464
AUC: 0.8371031746031746
PR AUC: 0.2831083593844067
AUC: 0.7463293650793651
PR AUC: 0.25696760035136806
AUC: 0.8511480601741884
PR 

In [54]:
with open("low_pride_pr_auc_w2v.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in pr_auc:
        csvwriter.writerow([row])    
    
with open("low_pride_auc_w2v.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in auc_scores:
        csvwriter.writerow([row])

# Pretrained Word2Vec


In [50]:
auc_scores = []
pr_auc = []

dat = pd.read_csv("20220131_all_paragraphs_2020_added_missings_added.csv")
annotated = pd.read_csv("All_annotated_data_round_1_low_pride_20220209.csv")
dat = dat.merge(annotated[["speech_par_id", "pred_class"]], how = "left", on = "speech_par_id")
avg_embeddings = loadtxt('avg_embeddings_pretrained.csv', delimiter=',')

for filename in os.listdir("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_low_pride/"):
    ## load the test set 
    file = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_low_pride/" + "/" + filename
    test = pd.read_csv(file)
    train = annotated[~annotated['speech_par_id'].isin(test["speech_par_id"])]
    test = test[test['speech_par_id'].isin(dat['speech_par_id'])]
    train = train[train['speech_par_id'].isin(dat['speech_par_id'])] # make sure annotated data and the original df match
# attach train/test set to original df
    train_id = train['speech_par_id']
    test_id = test['speech_par_id']    
    dat['test'] = np.where(dat['speech_par_id'].isin(test_id), 1, 0)
    dat['train'] = np.where(dat['speech_par_id'].isin(train_id), 1, 0)
    # get the embeddings to train/test set
    train_vec = np.asarray([avg_embeddings[i] for i in dat[dat['speech_par_id'].isin(train_id)].index.tolist()])
    test_vec = np.asarray([avg_embeddings[i] for i in dat[dat['speech_par_id'].isin(test_id)].index.tolist()])
    ## Fit the model to the training set
    text_clf = RandomForestClassifier(n_estimators=5000, max_depth=10, random_state=0, max_features = "sqrt", class_weight="balanced")
    text_clf.fit(train_vec,  dat[dat['train']==1].pred_class) # use the original df to match the order
    ## Predict out-of-sample on the test set and compute AUC
    preds = text_clf.predict_proba(test_vec)
    fpr_d2v, tpr_d2v, thresholds_d2v = metrics.roc_curve(dat[dat['test']==1].pred_class, preds[:,1], pos_label=1)
    auc_scores = auc_scores + [metrics.auc(fpr_d2v, tpr_d2v)]
    #PR AUC
    lr_precision, lr_recall, _ = precision_recall_curve(dat[dat['test']==1].pred_class,  preds[:,1].tolist())
    lr_auc = auc(lr_recall, lr_precision)
    pr_auc = pr_auc + [lr_auc]
    print("AUC: "+str(metrics.auc(fpr_d2v, tpr_d2v)))
    print("PR AUC: " + str(lr_auc))
    
print("Mean AUC: " + str(np.mean(auc_scores)))
print("Mean PR AUC: " + str(np.mean(pr_auc)))

confusion = confusion_matrix(dat[dat['test']==1].pred_class,text_clf.predict(test_vec) )
print(confusion)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AUC: 0.8417458432304038
PR AUC: 0.340082054419441
AUC: 0.8682539682539683
PR AUC: 0.34267495235981443
AUC: 0.8269841269841269
PR AUC: 0.21849476098478235
AUC: 0.7945436507936509
PR AUC: 0.28551953481772424
AUC: 0.869061757719715
PR AUC: 0.3218574614774693
AUC: 0.8743072050673001
PR AUC: 0.4214816086846277
AUC: 0.8776722090261282
PR AUC: 0.23233452505245822
AUC: 0.8431314330958036
PR AUC: 0.2541904344864129
AUC: 0.8742082343626286
PR AUC: 0.4499977901374089
AUC: 0.7845407759303247
PR AUC: 0.19764987608130694
AUC: 0.8135391923990499
PR AUC: 0.23720117189025916
AUC: 0.8538202692003167
PR AUC: 0.3758590764816222
AUC: 0.7991884402216943
PR AUC: 0.2599769211047365
AUC: 0.8520387965162312
PR AUC: 0.3258949339537761
AUC: 0.785827395091053
PR AUC: 0.2126309874651194
AUC: 0.8657957244655582
PR AUC: 0.3761872670199383
AUC: 0.7966269841269841
PR AUC: 0.26029777479078137
AUC: 0.8137896825396825
PR AUC: 0.3656808477689209
AUC: 0.7841269841269841
PR AUC: 0.2978480916390276
AUC: 0.8406571654790183
PR 

In [51]:
with open("low_pride_pr_auc_pretrained.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in pr_auc:
        csvwriter.writerow([row])    
    
with open("low_pride_auc_pretrained.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in auc_scores:
        csvwriter.writerow([row])

# Doc2Vec RF

In [43]:
auc_scores_d2v = []
pr_auc = []
accuracy_scores_d2v = []

dat = pd.read_csv("20201115_all_paragraphs.csv")
dat['speech_par_id'] = dat['Speech_id'].astype(str) + "_" + dat['par_id'].astype(str)

model = Word2Vec.load("doc2vec_wordvecs.model") 
annotated = pd.read_csv("All_annotated_data_round_1_low_pride_20220209.csv")
dat = dat.merge(annotated[["speech_par_id", "pred_class"]], how = "left", on = "speech_par_id")

for filename in os.listdir("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_low_pride/"):
    ## load the test set 
    file = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_low_pride/" + "/" + filename
    test = pd.read_csv(file)
    train = annotated[~annotated['speech_par_id'].isin(test["speech_par_id"])]
    test = test[test['speech_par_id'].isin(dat['speech_par_id'])]
    train = train[train['speech_par_id'].isin(dat['speech_par_id'])] # make sure annotated data and the original df match
    # merge test and train sets back onto the total df
    dat['test'] = np.where(dat['speech_par_id'].isin(test["speech_par_id"]), 1, 0)
    dat['train'] = np.where(dat['speech_par_id'].isin(train["speech_par_id"]), 1, 0)
    # so that teh order of rows are the same with the embeddings
    test_set = np.asarray([model.docvecs[i] for i in dat[dat['test'] == 1].index.tolist()])
    train_set = np.asarray([model.docvecs[i] for i in dat[dat['train']== 1].index.tolist()])
    ## Initialize a random forest classifier
    gbc = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=0, max_features = "sqrt", class_weight="balanced")
    ## Fit the model to the training set
    gbc.fit(train_set, dat[dat['train']==1].pred_class)
    ## Predict out-of-sample on the test set and compute AUC
    preds = gbc.predict_proba(test_set)
    fpr_d2v, tpr_d2v, thresholds_d2v = metrics.roc_curve(dat[dat['test']==1].pred_class, preds[:,1], pos_label=1)
    auc_scores_d2v = auc_scores_d2v + [metrics.auc(fpr_d2v, tpr_d2v)]
    #PR AUC
    lr_precision, lr_recall, _ = precision_recall_curve(dat[dat['test']==1].pred_class,  preds[:,1].tolist())
    lr_auc = auc(lr_recall, lr_precision)
    pr_auc = pr_auc + [lr_auc]
    print("AUC: "+str(metrics.auc(fpr_d2v, tpr_d2v)))
    accuracy_d2v = metrics.accuracy_score(dat[dat['test']==1].pred_class, gbc.predict(test_set))
    accuracy_scores_d2v = accuracy_scores_d2v + [accuracy_d2v]
    print("Accuracy: " + str(accuracy_d2v))
    print("PR_AUC: " + str(lr_auc))

print("Mean AUC: " + str(np.mean(auc_scores_d2v)))
print("Mean Accuracy: " + str(np.mean(accuracy_scores_d2v)))

confusion = confusion_matrix(dat[dat['test']==1].pred_class, gbc.predict(test_set))
print(confusion)

AUC: 0.8477830562153602
Accuracy: 0.946067415730337
PR_AUC: 0.29798266007344465
AUC: 0.7546626984126984
Accuracy: 0.9459459459459459
PR_AUC: 0.25004884692263635
AUC: 0.7875894988066825
Accuracy: 0.945823927765237
PR_AUC: 0.2948317954081235
AUC: 0.7618337311058075
Accuracy: 0.945823927765237
PR_AUC: 0.16610900034432255
AUC: 0.8149247822644496
Accuracy: 0.946067415730337
PR_AUC: 0.22014143810201348
AUC: 0.7526722090261282
Accuracy: 0.946067415730337
PR_AUC: 0.13345744300941714
AUC: 0.7855158730158731
Accuracy: 0.9459459459459459
PR_AUC: 0.20166848519952504
AUC: 0.870249406175772
Accuracy: 0.946067415730337
PR_AUC: 0.4493529028636205
AUC: 0.8214568487727633
Accuracy: 0.946067415730337
PR_AUC: 0.22321908068681476
AUC: 0.7584125098970704
Accuracy: 0.946067415730337
PR_AUC: 0.23834850114260503
AUC: 0.7821428571428571
Accuracy: 0.9459459459459459
PR_AUC: 0.2628000584783819
AUC: 0.8357086302454474
Accuracy: 0.946067415730337
PR_AUC: 0.20471283512235255
AUC: 0.7922604908946952
Accuracy: 0.94606

In [46]:
with open("low_pride_d2v_pr_auc.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in pr_auc:
        csvwriter.writerow([row])
    
with open("low_pride_d2v_auc.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in auc_scores_d2v:
        csvwriter.writerow([row])

In [44]:
print("Mean ROC-AUC: " + str(np.mean(auc_scores_d2v)))
print("Mean Accuracy: " + str(np.mean(accuracy_scores_d2v)))
print("Mean PR_AUC: " + str(np.mean(pr_auc)))


Mean ROC-AUC: 0.7984456534864998
Mean Accuracy: 0.945984750469393
Mean PR_AUC: 0.25684923521336733


# balanced RFC

In [47]:
auc_scores_balanced = []
pr_auc_balanced = []
accuracy_scores_balanced = []

dat = pd.read_csv("20201115_all_paragraphs.csv")
dat['speech_par_id'] = dat['Speech_id'].astype(str) + "_" + dat['par_id'].astype(str)
# X = np.asarray([model.docvecs[i] for i in label_dat.index.tolist()])
# Y = np.asarray(label_dat['label'].tolist(), dtype="int")
model = Word2Vec.load("doc2vec_wordvecs.model") 
dat = dat.merge(annotated[["speech_par_id", "pred_class"]], how = "left", on = "speech_par_id")


for filename in os.listdir("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_low_pride/"):
    ## load the test set 
    annotated = pd.read_csv("All_annotated_data_round_1_low_pride_20220209.csv")
    file = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_low_pride/" + "/" + filename
    test = pd.read_csv(file)
    train = annotated[~annotated['speech_par_id'].isin(test["speech_par_id"])]
    test = test[test['speech_par_id'].isin(dat['speech_par_id'])]
    train = train[train['speech_par_id'].isin(dat['speech_par_id'])] # make sure annotated data and the original df match
    # merge test and train sets back onto the total df
    dat['test'] = np.where(dat['speech_par_id'].isin(test["speech_par_id"]), 1, 0)
    dat['train'] = np.where(dat['speech_par_id'].isin(train["speech_par_id"]), 1, 0)
    # so that teh order of rows are the same with the embeddings
    test_set = np.asarray([model.docvecs[i] for i in dat[dat['test'] == 1].index.tolist()])
    train_set = np.asarray([model.docvecs[i] for i in dat[dat['train']== 1].index.tolist()])
    ## Initialize a random forest classifier
    brfc = BalancedRandomForestClassifier(n_estimators=5000, max_depth=3, random_state=0, max_features = "sqrt")
    ## Fit the model to the training set
    brfc.fit(train_set, dat[dat['train']==1].pred_class)
    ## Predict out-of-sample on the test set and compute AUC
    preds = brfc.predict_proba(test_set)
    fpr, tpr, thresholds = metrics.roc_curve(dat[dat['test']==1].pred_class, preds[:,1], pos_label=1)
    auc_scores_balanced = auc_scores_balanced + [metrics.auc(fpr, tpr)]
    #PR AUC
    lr_precision, lr_recall, _ = precision_recall_curve(dat[dat['test']==1].pred_class,  preds[:,1].tolist())
    lr_auc = auc(lr_recall, lr_precision)
    pr_auc_balanced = pr_auc_balanced + [lr_auc]
    print("AUC: "+str(metrics.auc(fpr, tpr)))
    accuracy_balanced = metrics.accuracy_score(dat[dat['test']==1].pred_class, brfc.predict(test_set))
    accuracy_scores_balanced = accuracy_scores_balanced + [accuracy_balanced]
    print("Accuracy: " + str(accuracy_balanced))
    print("PR_AUC: " + str(lr_auc))

print("Mean AUC: " + str(np.mean(auc_scores_balanced)))
print("Mean Accuracy: " + str(np.mean(accuracy_scores_balanced)))

AUC: 0.8525336500395883
Accuracy: 0.6966292134831461
PR_AUC: 0.34895721705593585
AUC: 0.8099206349206349
Accuracy: 0.722972972972973
PR_AUC: 0.29771768877035487
AUC: 0.8299522673031026
Accuracy: 0.7042889390519187
PR_AUC: 0.4059061806659194
AUC: 0.7900755767700876
Accuracy: 0.7110609480812641
PR_AUC: 0.16247843063160616
AUC: 0.8185866983372921
Accuracy: 0.7078651685393258
PR_AUC: 0.21768531155269744
AUC: 0.7915676959619953
Accuracy: 0.750561797752809
PR_AUC: 0.15715766763653932
AUC: 0.8582341269841269
Accuracy: 0.7364864864864865
PR_AUC: 0.22746274123923202
AUC: 0.8624307205067301
Accuracy: 0.7303370786516854
PR_AUC: 0.41404618217774375
AUC: 0.8705463182897862
Accuracy: 0.7168539325842697
PR_AUC: 0.30127957795354526
AUC: 0.7896872525732382
Accuracy: 0.698876404494382
PR_AUC: 0.2858382265095982
AUC: 0.8111111111111111
Accuracy: 0.7094594594594594
PR_AUC: 0.27469317867091897
AUC: 0.8506532066508313
Accuracy: 0.7101123595505618
PR_AUC: 0.27088692308705115
AUC: 0.8195764053840063
Accuracy:

In [49]:
with open("low_pride_pr_auc_balanced.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in pr_auc_balanced:
        csvwriter.writerow([row])
    
with open("low_pride_auc_balanced.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in auc_scores_balanced:
        csvwriter.writerow([row])
    
      

In [48]:
print("Mean ROC-AUC: " + str(np.mean(auc_scores_balanced)))
print("Mean Accuracy: " + str(np.mean(accuracy_scores_balanced)))
print("Mean PR_AUC: " + str(np.mean(pr_auc_balanced)))

Mean ROC-AUC: 0.8241207438097811
Mean Accuracy: 0.7168683467244858
Mean PR_AUC: 0.27437334490730014


Found existing installation: neuralcoref 4.0
Uninstalling neuralcoref-4.0:
  Would remove:
    /opt/anaconda3/lib/python3.7/site-packages/neuralcoref-4.0.dist-info/*
    /opt/anaconda3/lib/python3.7/site-packages/neuralcoref/*
Proceed (y/n)? ^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


NameError: name 'y' is not defined